# Build the logbook for Hologram with empty or filters

- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab/IN2P3/CNES, DESC-LSST fullmember, behalf AuxTel Teal VERA C. Rubin observatory
- creation date : September 19th 2022
- update : September 21th 2022


Works directory per directory (day by day)

CCIN2P3:

- works with kernels **anaconda3_auxtel** (with libradtran) at CC
- works with kernel **python 3** locally

In [1]:
import sys
sys.path

['/pbs/throng/lsst/users/dagoret/desc/AuxTelComm/notebookccdm_gen3/runspectractor_standalone/2022_09',
 '',
 '/opt/conda/lib/python3.8/site-packages',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3/lib/python38.zip',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3/lib/python3.8',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3/lib/python3.8/lib-dynload',
 '/pbs/home/d/dagoret/.local/lib/python3.8/site-packages',
 '/pbs/throng/lsst/users/dagoret/desc/firecrown',
 '/pbs/throng/lsst/users/dagoret/desc/augur',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3/lib/python3.8/site-packages',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3/lib/python3.8/site-packages/Deprecated-1.2.11-py3.8.egg',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3/lib/python3.8/site-packages/docopt-0.6.2-py3.8.egg',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3/lib/python3.8/site-packages/mimeparse-0.1.3-py3.8.egg',
 '/sps/lsst/groups/auxtel/softs/utils/anaconda3/lib/python3.8/site-packages/rail-0.1.dev0-py3.8.eg

In [2]:
! ls /sps/lsst/groups/auxtel/data/2022/BG40~holo4_003/20220629

exposure_2022062900054_postisrccd.fits	exposure_2022062900325_postisrccd.fits
exposure_2022062900055_postisrccd.fits	exposure_2022062900327_postisrccd.fits
exposure_2022062900056_postisrccd.fits	exposure_2022062900328_postisrccd.fits
exposure_2022062900084_postisrccd.fits	exposure_2022062900359_postisrccd.fits
exposure_2022062900110_postisrccd.fits	exposure_2022062900360_postisrccd.fits
exposure_2022062900113_postisrccd.fits	exposure_2022062900362_postisrccd.fits
exposure_2022062900137_postisrccd.fits	exposure_2022062900388_postisrccd.fits
exposure_2022062900138_postisrccd.fits	exposure_2022062900389_postisrccd.fits
exposure_2022062900139_postisrccd.fits	exposure_2022062900414_postisrccd.fits
exposure_2022062900164_postisrccd.fits	exposure_2022062900415_postisrccd.fits
exposure_2022062900165_postisrccd.fits	exposure_2022062900416_postisrccd.fits
exposure_2022062900166_postisrccd.fits	exposure_2022062900446_postisrccd.fits
exposure_2022062900201_postisrccd.fits	exposure_2022062900449_po

# Select the combination DATE, FILTER, DISPERSER

In [3]:
DATE = '20220630'
filterdispersernames = ["empty~holo4_003","BG40~holo4_003","FELH0600~holo4_003","SDSSg~holo4_003",\
                        "empty~ronchi170lpmm","BG40~ronchi170lpmm","FELH0600~ronchi170lpmm","SDSSg~ronchi170lpmm",\
                       ]
filterdispersername = filterdispersernames[0]
version="v5.0"  # new orientation

# If position need to be specified add it in this dictionnary

In [4]:
# reconstructed / good /xpos /ypos

if version == "v5.0":
    info={\

          # with red filter
          'exposure_2022031600351_postisrccd.fits':(1,1,300.,1700),\
         }
        

# Imports

In [5]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os
import re

plt.rcParams["figure.figsize"] = (16,4)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [6]:
from astropy.io import fits

In [7]:
import imp
MODULE_EXTENSIONS = ('.py', '.pyc', '.pyo')

/tmp/ipykernel_5208/3547059130.py:1: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


In [8]:
def package_contents(package_name):
    file, pathname, description = imp.find_module(package_name)
    if file:
        raise ImportError('Not a package: %r', package_name)
    # Use a set because some may be both source and compiled.
    return set([os.path.splitext(module)[0]
        for module in os.listdir(pathname)
        if module.endswith(MODULE_EXTENSIONS)])

In [9]:
package_contents("spectractor")

{'__init__',
 '_version',
 'astrometry',
 'config',
 'logbook',
 'parameters',
 'tools'}

In [10]:
package_contents("spectractor/extractor")

{'__init__',
 'background',
 'chromaticpsf',
 'dispersers',
 'extractor',
 'images',
 'psf',
 'spectroscopy',
 'spectrum',
 'targets'}

# Initialisation

In [11]:
dir_logbooks="logbooks"

In [12]:
import os
if not os.path.exists(dir_logbooks):
    os.makedirs(dir_logbooks)

## Get inputs files and sort them according date and number

In [13]:
home=os.getenv("HOME")

In [14]:
image_path=filterdispersername
logbook_csv=os.path.join(dir_logbooks,"auxtellogbook_" + filterdispersername + "_"+ DATE +"_" + version + ".csv")
logbook_xlsx=os.path.join(dir_logbooks,"auxtellogbook_" + filterdispersername + "_" +DATE +"_" + version + ".xlsx")

# Path

In [15]:
HOSTCC=True
LAPTOP=False

In [16]:
# Set path depending on which computer running (according HOSTCC)
if HOSTCC:
    path_auxtel="/sps/lsst/groups/auxtel"
    path_spectractor=os.path.join(path_auxtel,"softs/github/desc/Spectractor")
    path_spectractor_config=os.path.join(path_spectractor,"config")
    path_images=os.path.join(path_auxtel,"data/2022/"+filterdispersername+"/"+DATE)

else:
    if not LAPTOP:
        path_auxtel="/Users/sylvie/DATA/AuxTelData2022"
        path_spectractor=os.path.join(path_auxtel,"/Users/sylvie/MacOSX/GitHub/LSST/Spectractor")
        path_spectractor_config=os.path.join(path_spectractor,"config")
        path_images=os.path.join(path_auxtel,"data/2022/"+filterdispersername+"/"+DATE)
        
    else:
        path_auxtel="/Users/dagoret/DATA/AuxTelData2022"
        path_spectractor=os.path.join(path_auxtel,"/Users/dagoret/MacOSX/GitHub/LSST/Spectractor")
        path_spectractor_config=os.path.join(path_spectractor,"config")
        path_images=os.path.join(path_auxtel,"data/2022/"+filterdispersername+"/"+DATE)
        

# Check the images exists

In [17]:
dir_images=path_images

In [18]:
all_files=os.listdir(dir_images)

In [19]:
all_files

['exposure_2022063000077_postisrccd.fits',
 'exposure_2022063000338_postisrccd.fits',
 'exposure_2022063000216_postisrccd.fits',
 'exposure_2022063000061_postisrccd.fits',
 'exposure_2022063000231_postisrccd.fits',
 'exposure_2022063000561_postisrccd.fits',
 'exposure_2022063000178_postisrccd.fits',
 'exposure_2022063000367_postisrccd.fits',
 'exposure_2022063000076_postisrccd.fits',
 'exposure_2022063000589_postisrccd.fits',
 'exposure_2022063000177_postisrccd.fits',
 'exposure_2022063000476_postisrccd.fits',
 'exposure_2022063000107_postisrccd.fits',
 'exposure_2022063000368_postisrccd.fits',
 'exposure_2022063000247_postisrccd.fits',
 'exposure_2022063000060_postisrccd.fits',
 'exposure_2022063000562_postisrccd.fits',
 'exposure_2022063000232_postisrccd.fits',
 'exposure_2022063000405_postisrccd.fits',
 'exposure_2022063000091_postisrccd.fits',
 'exposure_2022063000276_postisrccd.fits',
 'exposure_2022063000450_postisrccd.fits',
 'exposure_2022063000353_postisrccd.fits',
 'exposure_

In [20]:
all_selected_files = []
for filename in all_files:
    if re.search(".fits$",filename):
        all_selected_files.append(filename)
N=len(all_selected_files)

In [21]:
all_selected_files=np.array(all_selected_files)

### Sort files

In [22]:
def file_tag_forsorting(filename):
    #m=re.findall('^Cor_holo4_003_.*([0-9][0-9][0-9][0-9]-[0-9][0-9]-[0-9][0-9]_.*)[.]fits$',filename)
    m=re.findall('exposure.*_(.*)_postisrccd[.]fits$',filename)
    
    return m[0]

In [23]:
file_tag_forsorting('exposure_2022031700310_postisrccd.fits')

'2022031700310'

In [24]:
#all_tags = [ file_tag_forsorting(filename)[0] for filename in all_selected_files]
all_tags = [ file_tag_forsorting(filename) for filename in all_selected_files]
all_tags=np.array(all_tags)
all_tags

array(['2022063000077', '2022063000338', '2022063000216', '2022063000061',
       '2022063000231', '2022063000561', '2022063000178', '2022063000367',
       '2022063000076', '2022063000589', '2022063000177', '2022063000476',
       '2022063000107', '2022063000368', '2022063000247', '2022063000060',
       '2022063000562', '2022063000232', '2022063000405', '2022063000091',
       '2022063000276', '2022063000450', '2022063000353', '2022063000436',
       '2022063000292', '2022063000277', '2022063000521', '2022063000136',
       '2022063000420', '2022063000491', '2022063000106', '2022063000137',
       '2022063000261', '2022063000391', '2022063000157', '2022063000451',
       '2022063000492', '2022063000435', '2022063000121', '2022063000537',
       '2022063000337', '2022063000352', '2022063000122', '2022063000536',
       '2022063000046', '2022063000318', '2022063000262', '2022063000522',
       '2022063000158', '2022063000406', '2022063000092', '2022063000590',
       '2022063000390', '

In [25]:
indexes=np.argsort(all_tags)

In [26]:
sorted_files=all_selected_files[indexes]

In [27]:
sorted_files

array(['exposure_2022063000046_postisrccd.fits',
       'exposure_2022063000060_postisrccd.fits',
       'exposure_2022063000061_postisrccd.fits',
       'exposure_2022063000076_postisrccd.fits',
       'exposure_2022063000077_postisrccd.fits',
       'exposure_2022063000091_postisrccd.fits',
       'exposure_2022063000092_postisrccd.fits',
       'exposure_2022063000106_postisrccd.fits',
       'exposure_2022063000107_postisrccd.fits',
       'exposure_2022063000121_postisrccd.fits',
       'exposure_2022063000122_postisrccd.fits',
       'exposure_2022063000136_postisrccd.fits',
       'exposure_2022063000137_postisrccd.fits',
       'exposure_2022063000157_postisrccd.fits',
       'exposure_2022063000158_postisrccd.fits',
       'exposure_2022063000177_postisrccd.fits',
       'exposure_2022063000178_postisrccd.fits',
       'exposure_2022063000216_postisrccd.fits',
       'exposure_2022063000231_postisrccd.fits',
       'exposure_2022063000232_postisrccd.fits',
       'exposure_202

In [28]:
N=len(sorted_files)

# Build the logbook from the headers

In [29]:
idx=0
#ccdir='/sps/lsst/groups/auxtel/data/2021/holo/FlipCleans'
ccdir=dir_images
all_dates=[]
all_files=[]
all_seqnum=[]
all_subdir=[]
all_indexes=[]
all_targets=[]
all_filters=[]
all_disperser=[]
all_airmass=[]
all_exposures=[]
all_focus=[]
all_seeing=[]
all_P=[]
all_T=[]
all_H=[]
all_W=[]
for filename in sorted_files:
    fullfilename=os.path.join(dir_images,filename)
    hdu = fits.open(fullfilename)
    hdu0=hdu[0]
    hdr0=hdu0.header 
    all_indexes.append(idx)
    all_seqnum.append(hdr0['SEQNUM'])
    all_files.append(filename)
    all_subdir.append(ccdir)
    targetname=hdr0['OBJECT']
    all_targets.append(targetname)
    all_dates.append(hdr0['DATE-OBS'])
    all_airmass.append(hdr0['AMSTART'])
    all_filters.append(hdr0['FILTER'])
    all_disperser.append(hdr0['GRATING'])
    all_exposures.append(hdr0['EXPTIME'])
    all_focus.append(hdr0['FOCUSZ'])
    all_seeing.append(hdr0['SEEING'])
    all_P.append(hdr0['PRESSURE'])
    all_T.append(hdr0['AIRTEMP'])
    all_H.append(hdr0['HUMIDITY'])
    all_W.append(hdr0['WINDSPD'])
       
    
    idx+=1
    

# Make pandas dataframe

## Create pandas dataframe

In [30]:
df=pd.DataFrame()

In [31]:
len(all_airmass)

57

In [32]:
N=len(all_indexes)

## Fill with container info

In [33]:
df['index']=all_indexes
df['date']=all_dates
df['path']=all_subdir
df['file']=all_files
df['num']=all_seqnum
df['object']=all_targets
df['filter']=all_filters
df['disperser']=all_disperser
df['airmass']=all_airmass
df['exposure']=all_exposures
df['focus']=all_focus
df['seeing']=all_seeing
df['pressure']=all_P
df['temperature']=all_T
df['humidity']=all_H
df['windspd']=all_W

## Add dictionnary info

### Initialise new columns

In [34]:
Obj_posXpix=np.full(N,np.nan)
Obj_posYpix=np.full(N,np.nan)
Dx=np.full(N,np.nan)
Dy=np.full(N,np.nan)
run=np.full(N,np.nan)
quality=np.full(N,np.nan)
skip=np.full(N,"    ")

### Fill dictionnary info

In [35]:
# loop on rows index
for idx in np.arange(N):
    filename=all_files[idx]
    # check if filename is in dictionnary
    if filename in info.keys():
        (run_flag,quality_flag,xpos,ypos)=info[filename]
        Obj_posXpix[idx]=int(xpos)
        Obj_posYpix[idx]=int(ypos)
        run[idx]=run_flag
        if quality_flag==1:
            skip[idx] = "    "
            quality[idx]=1
        else:
            skip[idx] = "skip"
            quality[idx]=0

In [36]:
df['Obj-posXpix']=Obj_posXpix
df['Obj-posYpix']=Obj_posYpix
df['Dx']=Dx
df['Dy']=Dy
df['skip']=skip
df['run']=run
df['quality']=quality

In [37]:
pd.set_option('display.max_rows', None)

In [38]:
df

,index,date,path,file,num,object,filter,disperser,airmass,exposure,...,temperature,humidity,windspd,Obj-posXpix,Obj-posYpix,Dx,Dy,skip,run,quality
0,0,2022-06-30T23:15:22.567,/sps/lsst/groups/auxtel/data/2022/empty~holo4_...,exposure_2022063000046_postisrccd.fits,46,HD142331,empty_1,holo4_003,1.403795,30.0,...,7.2,20.0,4.9,NaN,NaN,NaN,NaN,,NaN,NaN
1,1,2022-06-30T23:24:31.859,/sps/lsst/groups/auxtel/data/2022/empty~holo4_...,exposure_2022063000060_postisrccd.fits,60,HD142331,empty_1,holo4_003,1.361241,30.0,...,7.2,21.0,4.1,NaN,NaN,NaN,NaN,,NaN,NaN
2,2,2022-06-30T23:25:14.943,/sps/lsst/groups/auxtel/data/2022/empty~holo4_...,exposure_2022063000061_postisrccd.fits,61,HD142331,empty_1,holo4_003,1.358098,30.0,...,7.2,21.0,5.8,NaN,NaN,NaN,NaN,,NaN,NaN
3,3,2022-06-30T23:35:50.719,/sps/lsst/groups/auxtel/data/2022/empty~holo4_...,exposure_2022063000076_postisrccd.fits,76,HD185975,empty_1,holo4_003,2.024822,30.0,...,7.2,21.0,5.1,NaN,NaN,NaN,NaN,,NaN,NaN
4,4,2022-06-30T23:36:33.674,/sps/lsst/groups/auxtel/data/2022/empty~holo4_...,exposure_2022063000077_postisrccd.fits,77,HD185975,empty_1,holo4_003,2.024353,30.0,...,7.1,21.0,5.2,NaN,NaN,NaN,NaN,,NaN,NaN
5,5,2022-06-30T23:46:52.409,/sps/lsst/groups/auxtel/data/2022/empty~holo4_...,exposure_2022063000091_postisrccd.fits,91,HD142331,empty_1,holo4_003,1.274707,30.0,...,7.0,21.0,5.4,NaN,NaN,NaN,NaN,,NaN,NaN
6,6,2022-06-30T23:47:35.500,/sps/lsst/groups/auxtel/data/2022/empty~holo4_...,exposure_2022063000092_postisrccd.fits,92,HD142331,empty_1,holo4_003,1.272288,30.0,...,7.0,21.0,5.1,NaN,NaN,NaN,NaN,,NaN,NaN
7,7,2022-06-30T23:56:45.653,/sps/lsst/groups/auxtel/data/2022/empty~holo4_...,exposure_2022063000106_postisrccd.fits,106,HD142331,empty_1,holo4_003,1.243106,30.0,...,6.9,22.0,3.6,NaN,NaN,NaN,NaN,,NaN,NaN
8,8,2022-06-30T23:57:28.696,/sps/lsst/groups/auxtel/data/2022/empty~holo4_...,exposure_2022063000107_postisrccd.fits,107,HD142331,empty_1,holo4_003,1.240955,30.0,...,6.8,22.0,5.3,NaN,NaN,NaN,NaN,,NaN,NaN
9,9,2022-07-01T00:06:37.301,/sps/lsst/groups/auxtel/data/2022/empty~holo4_...,exposure_2022063000121_postisrccd.fits,121,HD142331,empty_1,holo4_003,1.215102,30.0,...,6.7,22.0,6.1,NaN,NaN,NaN,NaN,,NaN,NaN


# Make logbook


In [39]:
N=len(df)
N

57

In [40]:
if N>0:
    df.to_excel(logbook_xlsx)
    df.to_csv(logbook_csv)

In [41]:
logbook_csv

'logbooks/auxtellogbook_empty~holo4_003_20220630_v5.0.csv'